Let's start with the problem from coursera! Below, I've copied the ciphertext from the problem statement. It's also given that the maximum possible key length is 13.

In [1]:
c = 'F96DE8C227A259C87EE1DA2AED57C93FE5DA36ED4EC87EF2C63AAE5B9A7EFFD673BE4ACF7BE8923CAB1ECE7AF2DA3DA44FCF7AE29235A24C963FF0DF3CA3599A70E5DA36BF1ECE77F8DC34BE129A6CF4D126BF5B9A7CFEDF3EB850D37CF0C63AA2509A76FF9227A55B9A6FE3D720A850D97AB1DD35ED5FCE6BF0D138A84CC931B1F121B44ECE70F6C032BD56C33FF9D320ED5CDF7AFF9226BE5BDE3FF7DD21ED56CF71F5C036A94D963FF8D473A351CE3FE5DA3CB84DDB71F5C17FED51DC3FE8D732BF4D963FF3C727ED4AC87EF5DB27A451D47EFD9230BF47CA6BFEC12ABE4ADF72E29224A84CDF3FF5D720A459D47AF59232A35A9A7AE7D33FB85FCE7AF5923AA31EDB3FF7D33ABF52C33FF0D673A551D93FFCD33DA35BC831B1F43CBF1EDF67F0DF23A15B963FE5DA36ED68D378F4DC36BF5B9A7AFFD121B44ECE76FEDC73BE5DD27AFCD773BA5FC93FE5DA3CB859D26BB1C63CED5CDF3FE2D730B84CDF3FF7DD21ED5ADF7CF0D636BE1EDB79E5D721ED57CE3FE6D320ED57D469F4DC27A85A963FF3C727ED49DF3FFFDD24ED55D470E69E73AC50DE3FE5DA3ABE1EDF67F4C030A44DDF3FF5D73EA250C96BE3D327A84D963FE5DA32B91ED36BB1D132A31ED87AB1D021A255DF71B1C436BF479A7AF0C13AA14794'
MAX_L = 13

Let's do some preprocessing on the ciphertext to get an array of bytes:

In [2]:
import re
import numpy as np

(c := np.array([int(x, 16) for x in re.findall('..', c)], dtype=np.uint8))

array([249, 109, 232, 194,  39, 162,  89, 200, 126, 225, 218,  42, 237,
        87, 201,  63, 229, 218,  54, 237,  78, 200, 126, 242, 198,  58,
       174,  91, 154, 126, 255, 214, 115, 190,  74, 207, 123, 232, 146,
        60, 171,  30, 206, 122, 242, 218,  61, 164,  79, 207, 122, 226,
       146,  53, 162,  76, 150,  63, 240, 223,  60, 163,  89, 154, 112,
       229, 218,  54, 191,  30, 206, 119, 248, 220,  52, 190,  18, 154,
       108, 244, 209,  38, 191,  91, 154, 124, 254, 223,  62, 184,  80,
       211, 124, 240, 198,  58, 162,  80, 154, 118, 255, 146,  39, 165,
        91, 154, 111, 227, 215,  32, 168,  80, 217, 122, 177, 221,  53,
       237,  95, 206, 107, 240, 209,  56, 168,  76, 201,  49, 177, 241,
        33, 180,  78, 206, 112, 246, 192,  50, 189,  86, 195,  63, 249,
       211,  32, 237,  92, 223, 122, 255, 146,  38, 190,  91, 222,  63,
       247, 221,  33, 237,  86, 207, 113, 245, 192,  54, 169,  77, 150,
        63, 248, 212, 115, 163,  81, 206,  63, 229, 218,  60, 18

Now, first of all, we want to obtain the length $L$ of the key. 
We do this by, for each length $1 \leq \ell \leq 13$, taking every $\ell^{\text{th}}$ charater and calculating the sum of the squares of the frequency distribution of the character values (I call this the $\textbf{score})$.
When we have an incorrect key value for $\ell$, the frequencies should be approximately uniformly distributed.
If $\ell = L$, this distribution more closely resembles a permutated version of the average letter frequencies in normal English text, which will give a much higher score.

In [3]:
from collections import Counter

def crack_length(ciphertext):
    arr = []
    for l in range(1, MAX_L+1):
        stream = ciphertext[::l]                                        # Get every l'th character
        freq = np.array(list(Counter(stream).values()))/len(stream)     # Compute frequencies
        score = sum(freq**2)
        arr.append(score)
        
    return np.argmax(arr)+1, arr                                        # Return the l with the highest score

L, arr = crack_length(c)
L

7

Ofcourse, if we test an $\ell$ that's a multiple of $L$, all the testes characters are still shifted by the same amount by the cipher, so we'd also get a high score.
Hence, we still have to check that the actual key length isn't actually a divisor of the one we obtained from this function.
The only proper divisor of $7$ is $1$, and as we see below, $\ell = 1$ doesn't get a high score, so we can be reasonably certain that $L = 7$.

In [4]:
print('\n'.join(map(str,enumerate(arr, 1))))

(1, 0.013535536441828847)
(2, 0.013852421910366666)
(3, 0.019108280254777094)
(4, 0.017380063200229834)
(5, 0.024445450430058833)
(6, 0.0197083800672969)
(7, 0.08607266435986162)
(8, 0.024992818155702378)
(9, 0.029547881808472735)
(10, 0.0312358533272974)
(11, 0.03623580313683073)
(12, 0.03000000000000002)
(13, 0.043097151205259344)


We're ready to crack the key completely! In $p$, I store the distribution of letter frequencies in normal English text as found on wikipedia (https://en.wikipedia.org/wiki/Letter_frequency).

In [5]:
p = np.array([
    8.2, 1.5, 2.8, 4.3, 13, 2.2, 2, 6.1, 7, .15, .77, 4, 2.4, 6.7,
    7.5, 1.9, .095, 6, 6.3, 9.1, 2.8, .98, 2.4, .15, 2, .074
]) / 100 

Then, we crack the key one position $i$ at a time by taking the "stream" of characters that would have been shifted with $\texttt{key[i]}$, the key value at position $i$ and trying to shift it back with all possible byte values.
The resulting stream of plaintext should only contain ASCII values and have approximately the same distribution as $p$, so we check if the potential plaintext has ASCII values only and then compute 
$$\begin{equation}\sum_{\alpha \in \{a,b,c,\dots\}} p_\alpha q_\alpha\tag{1}\end{equation}$$
where $p_\alpha$ is the frequency of the letter $\alpha$ in normal English text and $q_\alpha$ is the frequency in the obtained potential plaintext.
The potential plaintext with the highest value for (1) is the most likely candidate for the original plaintext, so the byte value that gives it is the most likely value for $\texttt{key[i]}$.

In [6]:
def crack_key(ciphertext, L):
    key = np.empty(L, dtype=np.uint8)
    
    for i in range(L):
        stream = ciphertext[i::L]                      # Take every L'th character, starting at i
        best = 0
        for b in range(0x100):
            plain = stream ^ b                         # If b is correct, c = m ^ b, so c ^ b = m ^ b ^ b = m              
            
            if ((plain < 32) | (plain > 127)).any():   # Check for non-ASCII values
                continue
                
            letters = np.bincount(plain, minlength=123)[97:123]
                
            q = letters/sum(letters)
            if (score := sum(p*q)) > best:
                best = score
                key[i] = b

    return key

key = crack_key(c, L)

<ipython-input-6-c49c74aea3c8>:15: RuntimeWarning: invalid value encountered in true_divide
  q = letters/sum(letters)


To check our key, let's write a function that decrypts the entire ciphertext and use it with our found key.

In [7]:
def decrypt(ciphertext, key):
    return ''.join(chr(x ^ key[i % len(key)]) for i,x in enumerate(ciphertext))

decrypt(c, key)

'C7<p1*gr$5h<eise1h epr$&t,&e $+de6tu!< *# t &h+,qu 6 #*r,e$m*+g *1h 7 t-,n"6, 6 c07e &*m(0ni&$t,*n ,+ 1-e 57e6 nc eo#eat1$c. rskeC7<pt*"r$5hye-a6ebe + 06ede#o7ehu+!r !s,e,fe+ote1h*0sa+!sieofe<e$7s,e\'u1etr$!i1,on$) &7yp1*s<6te(6 2 ree!e6,gn ! $+d  3a)0at ! ,+ ae#a,7lye$de-oce(a++erkeF*7 e=$m5)e,e1h eVi" n 7e  +c7<pt,*ne6ch (ee2ase1h*0gh1et*ebee6e&0ree#o7ede&$d 6 a#1e7eite2a6ein3 n1 d,e\'u1ewee+o2ekn*2,e$nde1h,6 e= r&,see!e(*ns17a1 s,e1h$1 i1ec$+ b eb7*ke+ev 7y  $s,)y.'

Ohno, that doesn't look like a very sensible message :(
Let's isolate the decrypted streams and have a look at them:

In [8]:
for i in range(L):
    print(i, decrypt(c[i::L], [key[i]]))

0 Crsr utu, t  i  ctstyedu,taf,rnpeen taycre,i thsheeeatn,endees,ibe .
1 7$e$$!  e*-6&&,5 1k*e e+ee+ee$$1(e   eeek=e" , e1ee&#e3ee*e=e1e1 + 
2 <51&+<&6$1, *$+7e$e"-+#!,1!<'!)*6!!3!#$(e$1 +*(1e6#$12 '+21 !71eee$
3 phhtd h mhncmt eocCra orfhseui s e a adaFmhncnehteodeanuo,hreahcbvs
4 1< ,e*+#* "0(,16#.7$607 e*i$11&<26$),,e+*5  7ee**&7 76112e,&(1$$7 ,
5 *ee&6#,*+7670*- e <5e6e!+0e7e,76 ,+0+7-+7)e7<620e0e6ee ee$6,* 1+*7)
6 gipet qrg ,ennenarphbehsosostoytrgda loe eVepcagbrd iidwkn sns  kyy


Some streams look fine, but others have more special characters than letters.
When that happens, the score in $\texttt{crack_key}$ is taken over a very small amount of characters (only the lowercase letters), so we can get very high scores that aren't very significant.
To combat this, let's add in a threshold for the minimum amount of lowercase letters that should be present in a stream.
About 20% of English text is spaces and we want to allow some wiggle room for capital letters, punctuation, etc., so we'll demand that at least 60% of the stream should be lowercase letters.

In [9]:
def crack_key(c, L):
    key = np.empty(L, dtype=np.uint8)
    
    for i in range(L):
        stream = c[i::L]
        best = 0
        for b in range(0x100):
            plain = stream ^ b
            
            if ((plain < 32) | (plain > 127)).any():
                continue
                
            freq = np.bincount(plain, minlength=123)
            letters = freq[97:123]
            
            # Added check of what fraction of stream are lowercase letters
            if (tot := sum(letters)) < .6*sum(freq):
                continue
                
            q = letters/tot
            if (score := sum(p*q)) > best:
                best = score
                key[i] = b

    return key

Let's see whether this has helped us:

In [10]:
key = crack_key(c, L)
print(key)
decrypt(c, key)

[186  31 145 178  83 205  62]


'Cryptography is the practice and study of techniques for, among other things, secure communication in the presence of attackers. Cryptography has been used for hundreds, if not thousands, of years, but traditional cryptosystems were designed and evaluated in a fairly ad hoc manner. For example, the Vigenere encryption scheme was thought to be secure for decades after it was invented, but we now know, and this exercise demonstrates, that it can be broken very easily.'

Whoohoo! We have cracked the message!
We should have all the tools we need to crack this next message as well:

In [11]:
c = '4576C64965DEAF6D87706D830B5EE48239CF7070830045EB9C2C9C656CD60A5FF89C28CF6576C21D0BEC822180666D83005FAD9A22CF7870D70C59EE8B3D9B317FCF0444FE9A6D8A677BD1105FE58723883F3EF4005FE5CE3987786D830A4AFD8F2F867D77D71007AD9A258A3168C21A5FAD832C857E6CCA1D52AD812BCF796BCE0845AD8D22827C6BCD0048EC9A24807F6D830859E8CE2C9A6571CE085FE48D2C837D67830045EA8B3E9B747A831E42F986229A653ED70859EA8B39867F798D4962EBCE04CF667FCD1D4EE9CE3980316DC60C0BF481389D317BCE0842E19D6D80633EDA065EFFCE3A86777B841A0BFD8622817432830847E1CE04CF797FD50C0BF9816D8B7E3ECA1A0BF89D28CF7870D70C59EE8B3D9B623083200BEE8F23CF767BD74952E29B3FCF7473C20047FEC26D9F706DD01E44FF8A3EC3316ECB0645E8CE3F8A7271D10D58A1CE2E9D747ACA1D0BEE8F3F8B623083200BE98123C8653ED40845F9CE39803172CA1F4EAD8723CF703ED00648E48B3996316ACB085FAD8A228A623ED7014EFE8B6D9C7E6CD74944EBCE39877870C41A05A3C06DA6317ACC4945E29A6D987070D7495FE2CE2186677B830045AD8F6D987E6CCF0D0BFA86289D743EC61F4EFF9739877870C44962AD8A22CF7070C74958EC976D86623ED10C48E29C298A7530833D43EC9A6D86623ECD065FAD9D2282746ACB0045EACE04CF7073831E42E1822481763ED7060BFE9B3D9F7E6CD74944FFCE2186677B831C45E98B3FC1'
c = np.array([int(x, 16) for x in re.findall('..', c)], dtype=np.uint8)
L, arr = crack_length(c)
print(L)
print('\n'.join(map(str, enumerate(arr,1))))

9
(1, 0.012358950368446028)
(2, 0.013289256198347114)
(3, 0.028695989727970383)
(4, 0.015542953161100617)
(5, 0.0186776859504132)
(6, 0.028591682419659755)
(7, 0.023233456176894703)
(8, 0.022894349926486045)
(9, 0.07659231389411447)
(10, 0.02479338842975203)
(11, 0.030400000000000014)
(12, 0.038752362948960284)
(13, 0.028664142779881035)


No divisors of 9 have high scores, yey! We can continue with $L=9$.

In [12]:
key = crack_key(c, L)
print(key)
decrypt(c, key)

[ 17  30 163 105  43 139 238  77 239]


"The NUA has buolt an in`rastructsre that gllows it&to intereept almout everytning. Witn this cavability,&the vast&majority&of human&communicgtions arc automatocally inaested wirhout taraeting. I` I wanteb to see \x7four emaijs or yout wife's vhone, alj I have ro do is sse intereepts. I ean get yiur emailu, passwotds, phonc records* credit eards. I bon't wanr to live&in a socoety that&does theue sort o` things.(. I do nit want ti live in&a world qhere evetything I&do and sgy is recirded. Thgt is not&somethina I am wijling to uupport ot live unber."

That's almost right... It seems like one stream is off, let's isolate them and check:

In [13]:
for i in range(L):
    print(i, decrypt(c[i::L], [key[i]]))

0 Taattwi v.il ohmntletnw  rfehoisgea cest a soi avoeiasdseagov
1 hsnrhsnae sivruusoyd gase e,a n.emspod. l t rndner nn . tm re
2 e  ua tlrW taimn m  t.nemy' vit tashri wishttgot legdr nh tt 
3  bictiemyicystaiaaiwa teaosaeseI iwodtIavoah s  idv  eToiwo u
4 Nunt trotta,tyncrtnirIe iu l  r ylons  necteo.ntn eIschtni on
5 Uo`sg&eunnv&&&&gcoara`bjtvjrseeiutc*ebr&o&u`(ii&qt&gig&ajutb
6 Alrrlteti atmoct cehe  os h oseau,d  ao iede .t ahydyrts lu e
7  taelop ntbhafoiaasotItu woI epnr srcrntnto t  l eto d oIiplr
8 h s o teghiej moultui oroin d t  p,erd'o yeshIwiwrh ieim npi.


Channel 5 is the culprit >:(
Let's check all the byte values that gave a score of better than .05:

In [14]:
for b in range(0x100):
    plain = c[5::L] ^ b
    if ((plain < 32) | (plain > 127)).any():
        continue
    freq = np.bincount(plain, minlength=123)
    letters = freq[97:123]
    if (tot := sum(letters)) < .6*sum(freq):
        continue
    q = letters/tot
    if (score := sum(p*q)) > .05:
        print(b, score)

139 0.054387500000000005
141 0.054375


The best score is obtained by 139, but 141 is very close, so let's try that:

In [15]:
key[5] = 141
decrypt(c, key)

"The NSA has built an infrastructure that allows it to intercept almost everything. With this capability, the vast majority of human communications are automatically ingested without targeting. If I wanted to see your emails or your wife's phone, all I have to do is use intercepts. I can get your emails, passwords, phone records, credit cards. I don't want to live in a society that does these sort of things... I do not want to live in a world where everything I do and say is recorded. That is not something I am willing to support or live under."

Succes :)